In [119]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *



In [120]:
from pyspark.sql.functions import col, avg, sum, desc, corr, when

In [121]:
#Initialize Spark Session
spark = SparkSession.builder \
    .appName("Retail Sales Analysis") \
    .getOrCreate()

In [122]:
#Load the Dataset
file_path = "C:\BlinkIT Grocery Data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)


<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\hp\AppData\Local\Temp\ipykernel_3400\3887444253.py:2: SyntaxWarning: invalid escape sequence '\B'
  file_path = "C:\BlinkIT Grocery Data.csv"


In [123]:
#Data Preview
print("Dataset Schema:")
df.printSchema()

Dataset Schema:
root
 |-- Item Fat Content: string (nullable = true)
 |-- Item Identifier: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Outlet Establishment Year: integer (nullable = true)
 |-- Outlet Identifier: string (nullable = true)
 |-- Outlet Location Type: string (nullable = true)
 |-- Outlet Size: string (nullable = true)
 |-- Outlet Type: string (nullable = true)
 |-- Item Visibility: double (nullable = true)
 |-- Item Weight: double (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Rating: double (nullable = true)



In [124]:
print("Dataset Preview:")
df.show()

Dataset Preview:
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+
|Item Fat Content|Item Identifier|           Item Type|Outlet Establishment Year|Outlet Identifier|Outlet Location Type|Outlet Size|      Outlet Type|Item Visibility|Item Weight|   Sales|Rating|
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+
|         Regular|          FDX32|Fruits and Vegeta...|                     2012|           OUT049|              Tier 1|     Medium|Supermarket Type1|      0.1000135|       15.1|145.4786|   5.0|
|         Low Fat|          NCB42|  Health and Hygiene|                     2022|           OUT018|              Tier 3|     Medium|Supermarket Type2|    0.008596051|       11.8|115.3492|   5.0|
|       

In [125]:

# 1. Handle missing values in 'Item Weight'
df_cleaned = df.withColumn("Item Weight", when(col("Item Weight").isNull(), df.agg(avg("Item Weight")).first()[0]).otherwise(col("Item Weight")))


In [126]:


df_cleaned.show()

+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+------------------+--------+------+
|Item Fat Content|Item Identifier|           Item Type|Outlet Establishment Year|Outlet Identifier|Outlet Location Type|Outlet Size|      Outlet Type|Item Visibility|       Item Weight|   Sales|Rating|
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+------------------+--------+------+
|         Regular|          FDX32|Fruits and Vegeta...|                     2012|           OUT049|              Tier 1|     Medium|Supermarket Type1|      0.1000135|              15.1|145.4786|   5.0|
|         Low Fat|          NCB42|  Health and Hygiene|                     2022|           OUT018|              Tier 3|     Medium|Supermarket Type2|    0.008596051|              11.8|115.349

In [127]:
df

DataFrame[Item Fat Content: string, Item Identifier: string, Item Type: string, Outlet Establishment Year: int, Outlet Identifier: string, Outlet Location Type: string, Outlet Size: string, Outlet Type: string, Item Visibility: double, Item Weight: double, Sales: double, Rating: double]

In [128]:
df.show()

+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+
|Item Fat Content|Item Identifier|           Item Type|Outlet Establishment Year|Outlet Identifier|Outlet Location Type|Outlet Size|      Outlet Type|Item Visibility|Item Weight|   Sales|Rating|
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+
|         Regular|          FDX32|Fruits and Vegeta...|                     2012|           OUT049|              Tier 1|     Medium|Supermarket Type1|      0.1000135|       15.1|145.4786|   5.0|
|         Low Fat|          NCB42|  Health and Hygiene|                     2022|           OUT018|              Tier 3|     Medium|Supermarket Type2|    0.008596051|       11.8|115.3492|   5.0|
|         Regular|       

In [129]:
# 2. Average sales by Item Type
avg_sales_by_item = df_cleaned.groupBy("Item Type") \
    .agg(avg("Sales").alias("Average_Sales")) \
    .orderBy(desc("Average_Sales"))

In [152]:
print("Average Sales by Item Type:")
avg_sales_by_item.show()

Average Sales by Item Type:
+--------------------+------------------+
|           Item Type|     Average_Sales|
+--------------------+------------------+
|           Household|149.42475318681318|
|               Dairy|148.49921055718468|
|       Starchy Foods| 147.8380229729729|
|         Snack Foods|146.19493533333352|
|Fruits and Vegeta...|144.58123457792203|
|             Seafood|141.84171874999998|
|           Breakfast|141.78815090909092|
|              Breads|140.95266852589648|
|                Meat|139.88203247058823|
|              Canned| 139.7638351309707|
|        Frozen Foods|138.50336612149542|
|         Hard Drinks|137.07794672897194|
|              Others| 132.8514295857988|
|         Soft Drinks|131.49251011235953|
|  Health and Hygiene| 130.8189207692308|
|        Baking Goods|126.38076604938273|
+--------------------+------------------+



In [147]:
# 3. Total sales by Item Type
total_sales_by_item = df_cleaned.groupBy("Item Type") \
    .agg(sum("Sales").alias("Total_Sales")) \
    .orderBy(desc("Total_Sales"))

In [148]:
print("Total Sales by Item Type")
total_sales_by_item.show()

Total Sales by Item Type
+--------------------+------------------+
|           Item Type|       Total_Sales|
+--------------------+------------------+
|Fruits and Vegeta...|178124.08099999995|
|         Snack Foods| 175433.9224000002|
|           Household|135976.52539999998|
|        Frozen Foods|118558.88140000009|
|               Dairy|101276.46159999995|
|              Canned| 90706.72899999999|
|        Baking Goods| 81894.73640000001|
|  Health and Hygiene| 68025.83880000001|
|                Meat| 59449.86379999999|
|         Soft Drinks| 58514.16699999999|
|              Breads|35379.119800000015|
|         Hard Drinks|29334.680599999996|
|              Others|        22451.8916|
|       Starchy Foods| 21880.02739999999|
|           Breakfast|15596.696600000001|
|             Seafood| 9077.869999999999|
+--------------------+------------------+



In [131]:
# 4. Total sales by Outlet Type
total_sales_by_outlet = df_cleaned.groupBy("Outlet Type") \
    .agg(sum("Sales").alias("Total_Sales")) \
    .orderBy(desc("Total_Sales"))

In [132]:
print("Total Sales by Outlet Type:")
total_sales_by_outlet.show()

Total Sales by Outlet Type:
+-----------------+------------------+
|      Outlet Type|       Total_Sales|
+-----------------+------------------+
|Supermarket Type1| 787549.8928000013|
|    Grocery Store|        151939.149|
|Supermarket Type2|131477.77639999994|
|Supermarket Type3|130714.67460000006|
+-----------------+------------------+



In [133]:
# 5. Correlation between Item Visibility and Sales
correlation = df_cleaned.stat.corr("Item Visibility", "Sales")
print(f"Correlation between Item Visibility and Sales: {correlation}")


Correlation between Item Visibility and Sales: -0.001314959837573848


In [138]:
# 6. Outlets with the highest average ratings
avg_ratings_by_outlet = df_cleaned.groupBy("Outlet Identifier") \
    .agg(avg("Rating").alias("Average_Rating")) \
    .orderBy(desc("Average_Rating"))

In [135]:
print("Top 5 Outlets with Highest Average Ratings:")
avg_ratings_by_outlet.show(5)

Top 5 Outlets with Highest Average Ratings:
+-----------------+------------------+
|Outlet Identifier|    Average_Rating|
+-----------------+------------------+
|           OUT019| 3.995075757575764|
|           OUT049|3.9854838709677347|
|           OUT017|3.9816414686825015|
|           OUT010|3.9771171171171207|
|           OUT018|3.9712284482758555|
+-----------------+------------------+
only showing top 5 rows

